In [1]:
import pandas as pd
import numpy as np

#
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Dropout, Input, Flatten
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
df = pd.read_csv("nyc-east-river-bicycle-crossings.zip")
df["Date"] = pd.to_datetime(df['Date'])
df["day_of_week"] = df["Date"].dt.day_name()

In [3]:
df.head()

,Unnamed: 0,Date,Day,High Temp (°F),Low Temp (°F),Precipitation,Brooklyn Bridge,Manhattan Bridge,Williamsburg Bridge,Queensboro Bridge,Total,day_of_week
0,0,2016-04-01,2016-04-01 00:00:00,78.1,66.0,0.01,1704.0,3126,4115.0,2552.0,11497,Friday
1,1,2016-04-02,2016-04-02 00:00:00,55.0,48.9,0.15,827.0,1646,2565.0,1884.0,6922,Saturday
2,2,2016-04-03,2016-04-03 00:00:00,39.9,34.0,0.09,526.0,1232,1695.0,1306.0,4759,Sunday
3,3,2016-04-04,2016-04-04 00:00:00,44.1,33.1,0.47 (S),521.0,1067,1440.0,1307.0,4335,Monday
4,4,2016-04-05,2016-04-05 00:00:00,42.1,26.1,0,1416.0,2617,3081.0,2357.0,9471,Tuesday


In [4]:
day_of_week_2_int = {}
for dow_idx, dow in enumerate(pd.unique(df['day_of_week'])):
    day_of_week_2_int[dow] = dow_idx

In [5]:
day_of_week_2_int

{'Friday': 0,
 'Monday': 3,
 'Saturday': 1,
 'Sunday': 2,
 'Thursday': 6,
 'Tuesday': 4,
 'Wednesday': 5}

In [6]:
df_encoded = df.copy()
df_encoded['day_of_week_encoded'] = df['day_of_week'].apply(lambda x: day_of_week_2_int[x])

In [7]:
df_encoded.head()

,Unnamed: 0,Date,Day,High Temp (°F),Low Temp (°F),Precipitation,Brooklyn Bridge,Manhattan Bridge,Williamsburg Bridge,Queensboro Bridge,Total,day_of_week,day_of_week_encoded
0,0,2016-04-01,2016-04-01 00:00:00,78.1,66.0,0.01,1704.0,3126,4115.0,2552.0,11497,Friday,0
1,1,2016-04-02,2016-04-02 00:00:00,55.0,48.9,0.15,827.0,1646,2565.0,1884.0,6922,Saturday,1
2,2,2016-04-03,2016-04-03 00:00:00,39.9,34.0,0.09,526.0,1232,1695.0,1306.0,4759,Sunday,2
3,3,2016-04-04,2016-04-04 00:00:00,44.1,33.1,0.47 (S),521.0,1067,1440.0,1307.0,4335,Monday,3
4,4,2016-04-05,2016-04-05 00:00:00,42.1,26.1,0,1416.0,2617,3081.0,2357.0,9471,Tuesday,4


In [8]:
df_encoded_selected = df_encoded[['day_of_week_encoded', 'Total', 'Brooklyn Bridge']]

In [9]:
df['day_of_week']

0         Friday
1       Saturday
2         Sunday
3         Monday
4        Tuesday
5      Wednesday
6       Thursday
7         Friday
8       Saturday
9         Sunday
10        Monday
11       Tuesday
12     Wednesday
13      Thursday
14        Friday
15      Saturday
16        Sunday
17        Monday
18       Tuesday
19     Wednesday
20      Thursday
21        Friday
22      Saturday
23        Sunday
24        Monday
25       Tuesday
26     Wednesday
27      Thursday
28        Friday
29      Saturday
         ...    
180       Friday
181     Saturday
182       Sunday
183       Monday
184      Tuesday
185    Wednesday
186     Thursday
187       Friday
188     Saturday
189       Sunday
190       Monday
191      Tuesday
192    Wednesday
193     Thursday
194       Friday
195     Saturday
196       Sunday
197       Monday
198      Tuesday
199    Wednesday
200     Thursday
201       Friday
202     Saturday
203       Sunday
204       Monday
205      Tuesday
206    Wednesday
207     Thursd

In [10]:
from sklearn import preprocessing

x = df_encoded_selected.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_train = pd.DataFrame(x_scaled)
df_train.set_index(df_encoded_selected.index)
df_train.columns = df_encoded_selected.columns

In [11]:
df_train.head()

In [12]:
embedding_size = 3

In [13]:
def build_model(input_length, output_length, embedding_size):
    model = Sequential()
    model.add(Embedding(input_dim = 7+1, output_dim = embedding_size, input_length = input_length, name="embedding"))
    model.add(Dense(units=40, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(units=10, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(units=output_length, name="targets"))
    
    return model

In [14]:
model = build_model(1, 2, embedding_size)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1, 3)              24        
_________________________________________________________________
dense (Dense)                (None, 1, 40)             160       
_________________________________________________________________
dropout (Dropout)            (None, 1, 40)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1, 10)             410       
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 10)             0         
_________________________________________________________________
flatten (Flatten)            (None, 10)                0         
_________________________________________________________________
targets (Dense)              (None, 2)                 22        
Total para

In [15]:
model.compile(loss='mean_squared_error', optimizer = "adam")

In [17]:
X_train = np.reshape(df_encoded_selected['day_of_week_encoded'].values, (-1, 1))        

In [18]:
y_train = np.reshape(df_encoded_selected[['Brooklyn Bridge', 'Total']].values, (-1, 2))

In [19]:
early_stopping_callback = EarlyStopping(monitor='loss', min_delta=1E-6, patience=10, verbose=0, mode='auto')

In [20]:
model.fit(x=X_train, y=y_train, 
          epochs = 200, 
          batch_size = 3,
          callbacks = [early_stopping_callback])

/home/ubuntu/.local/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/200
210/210 [==============================] - 3s 14ms/step - loss: 124112704.9143
Epoch 2/200
210/210 [==============================] - 0s 534us/step - loss: 124102304.3429
Epoch 3/200
210/210 [==============================] - 0s 530us/step - loss: 124062602.6000
Epoch 4/200
210/210 [==============================] - 0s 538us/step - loss: 123953223.2286
Epoch 5/200
210/210 [==============================] - 0s 541us/step - loss: 123683849.2571
Epoch 6/200
210/210 [==============================] - 0s 547us/step - loss: 123123155.7714
Epoch 7/200
210/210 [==============================] - 0s 537us/step - loss: 122138518.2571
Epoch 8/200
210/210 [==============================] - 0s 541us/step - loss: 120503088.7714
Epoch 9/200
210/210 [==============================] - 0s 541us/step - loss: 117736880.0571
Epoch 10/200
210/210 [==============================] - 0s 582us/step - loss: 114694530.2857
Epoch 11/200
210/210 [==============================] - 0s 624us/step - loss: 11

In [ ]:
model.layers[0].get_weights()

In [24]:
y_train

array([[ 1704., 11497.],
       [  827.,  6922.],
       [  526.,  4759.],
       [  521.,  4335.],
       [ 1416.,  9471.],
       [ 1885., 11919.],
       [ 1276.,  9596.],
       [ 1982., 12744.],
       [  504.,  4510.],
       [ 1447.,  9126.],
       [ 2005., 13312.],
       [ 1045.,  8067.],
       [ 2840., 18422.],
       [ 2861., 18315.],
       [ 2770., 17570.],
       [ 2384., 16375.],
       [ 3147., 18180.],
       [ 3871., 23090.],
       [ 3501., 23318.],
       [ 3450., 22591.],
       [ 3436., 22127.],
       [ 2975., 17837.],
       [ 2055., 13426.],
       [ 2798., 16740.],
       [ 3463., 20850.],
       [ 1997., 13005.],
       [ 3343., 19914.],
       [ 2486., 15631.],
       [ 2375., 14954.],
       [ 3199., 17432.],
       [ 1704., 11497.],
       [  827.,  6922.],
       [  526.,  4759.],
       [  521.,  4335.],
       [ 1416.,  9471.],
       [ 1885., 11919.],
       [ 1276.,  9596.],
       [ 1982., 12744.],
       [  504.,  4510.],
       [ 1447.,  9126.],
